In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [2]:

import os
from google.colab import drive

In [3]:
drive.mount('/content/drive')
pwd = "/content/drive/My Drive/IE434_ProjectGroup7/"
os.chdir(pwd)
rides_data=pd.read_pickle('Merged_Data.pkl')
rides_data['started_at']=pd.to_datetime(rides_data['started_at'])
rides_data['ended_at']=pd.to_datetime(rides_data['ended_at'])

Mounted at /content/drive


FileNotFoundError: ignored

In [ ]:
rides_data.sort_values(by=['started_at'],inplace=True)

In [ ]:
rides_data['member_casual'] = np.where((rides_data['member_casual'] == 'casual') | (rides_data['member_casual'] == 'Customer'), 'casual', rides_data['member_casual'])
rides_data['member_casual'] = np.where((rides_data['member_casual'] == 'member') | (rides_data['member_casual'] == 'Subscriber'), 'member', rides_data['member_casual'])

In [ ]:

rides_data=rides_data.drop(columns=['member_casual','distance','duration','DATE', 'TEMP', 'VISIB', 'WDSP', 'MAX', 'MIN', 'PRCP','start_date', 'start_time', 'end_date',
       'end_time'])

In [ ]:
rides_data.columns

## Preparing features from the raw data.

In [ ]:
rides_data.reset_index(drop=True)

In [ ]:
rides_data['end_station_name'].isna().value_counts()

In [ ]:
import pandas as pd

rides_data['started_at'] = pd.to_datetime(rides_data['started_at'])
rides_data['ended_at'] = pd.to_datetime(rides_data['ended_at'])

rides_data['day_of_week'] = rides_data['started_at'].dt.dayofweek
rides_data['year'] = rides_data['started_at'].dt.year
rides_data['hour_of_day'] = rides_data['started_at'].dt.hour
rides_data['month'] = rides_data['started_at'].dt.month

rides_data['date'] = rides_data['started_at'].dt.date


aggregated_data = pd.DataFrame()


outgoing_rides = rides_data.groupby(['start_station_name', 'date','year', 'month','day_of_week', 'hour_of_day']).size().reset_index(name='total_rides_out')
incoming_rides = rides_data.groupby(['end_station_name', 'date','year',  'month','day_of_week', 'hour_of_day']).size().reset_index(name='total_rides_in')


outgoing_rides.rename(columns={'start_station_name':'Station'},inplace=True)
incoming_rides.rename(columns={'end_station_name':'Station'},inplace=True)

aggregated_data = pd.merge(outgoing_rides, incoming_rides, left_on=['Station', 'date','year', 'month','day_of_week', 'hour_of_day'],right_on=['Station', 'date','year', 'month', 'day_of_week', 'hour_of_day'], how='outer')

aggregated_data['total_rides_out'] = aggregated_data['total_rides_out'].fillna(0)
aggregated_data['total_rides_in'] = aggregated_data['total_rides_in'].fillna(0)


aggregated_data['Bike_demand'] = aggregated_data['total_rides_out'] - aggregated_data['total_rides_in']


aggregated_data = aggregated_data[['Station', 'date','year',  'month','day_of_week', 'hour_of_day', 'total_rides_out', 'total_rides_in', 'Bike_demand']]

aggregated_data


In [ ]:
aggregated_data.sort_values(by=['date','hour_of_day'])

In [ ]:
aggregated_data.sort_values(by=['date'],ascending=True,inplace=True)
aggregated_data.reset_index(drop=True,inplace=True)
aggregated_data['Station'].nunique()

In [ ]:
hot_encoded_stations = pd.get_dummies(aggregated_data['Station'])
aggregated_data = pd.concat([aggregated_data, hot_encoded_stations], axis=1)
aggregated_data

In [ ]:
Y_out=aggregated_data[['date','year','total_rides_out']]
Y_in=aggregated_data[['date','year','total_rides_in']]
X=aggregated_data.drop(columns=['total_rides_out','total_rides_in','Station','Bike_demand'])

In [ ]:
X_test=X[X['year']>2022]
X_train=X[(X['year']<=2022)&(X['year']>2021)]

In [ ]:
y_test_out=Y_out[Y_out['year']>2022]
y_test_in=Y_in[Y_in['year']>2022]
y_train_out=Y_out[(Y_out['year']<=2022)&(Y_out['year']>2021)]
y_train_in=Y_in[(Y_in['year']<=2022)&(Y_in['year']>2021)]

In [ ]:
X_train.drop(columns=['date'],inplace=True)
y_train_out.drop(columns=['date','year'],inplace=True)
y_train_in.drop(columns=['date','year'],inplace=True)

In [ ]:
X_test.drop(columns=['date'],inplace=True)
y_test_out.drop(columns=['date','year'],inplace=True)
y_test_in.drop(columns=['date','year'],inplace=True)

In [ ]:
X_train

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_in_tensor = torch.tensor(y_train_in.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_in_tensor = torch.tensor(y_test_in.values, dtype=torch.float32)


In [ ]:

train_dataset = TensorDataset(X_train_tensor, y_train_in_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


## Implementing a simple feed forward network as the baseline model.

### The objective is to find the count of outgoing and incoming rides at all stations for every 1 hour timeslot.

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:

model = SimpleNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import torch
import matplotlib.pyplot as plt


train_losses = []
val_losses = []


epochs = 10
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())


    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test_tensor)
        val_loss = criterion(val_outputs, y_test_in_tensor)
        val_losses.append(val_loss.item())

y_pred_in = val_outputs.detach().numpy()



In [ ]:
y_train_out_tensor = torch.tensor(y_train_out.values, dtype=torch.float32)

y_test_out_tensor = torch.tensor(y_test_out.values, dtype=torch.float32)


In [ ]:
train_dataset = TensorDataset(X_train_tensor, y_train_out_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [ ]:
model = SimpleNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import torch
import matplotlib.pyplot as plt


train_losses_out = []
val_losses_out = []


epochs = 10
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_losses_out.append(loss.item())


    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test_tensor)
        val_loss = criterion(val_outputs, y_test_in_tensor)
        val_losses_out.append(val_loss.item())

y_pred_out = val_outputs.detach().numpy()

## Calculation of Loss and Metric of the Baseline Model

In [ ]:
rmse_in = mean_squared_error(y_test_in_tensor.numpy(), y_pred_in, squared=False)
mae_in = mean_absolute_error(y_test_in_tensor.numpy(), y_pred_in)


rmse_out = mean_squared_error(y_test_out_tensor.numpy(), y_pred_out, squared=False)
mae_out = mean_absolute_error(y_test_out_tensor.numpy(), y_pred_out)


print(f"RMSE (y_test_in): {rmse_in}")
print(f"MAE (y_test_in): {mae_in}")

print(f"RMSE (y_test_out): {rmse_out}")
print(f"MAE (y_test_out): {mae_out}")

In [ ]:
# Function to calculate feature importance using random permutations
def calculate_feature_importance(model, X, y_true, feature_names, criterion, n_permutations=30):
    baseline_loss = criterion(model(X), y_true).item()
    feature_importance = []

    for idx, feature in enumerate(feature_names):
        permuted_X = X.clone()
        permuted_X[:, idx] = torch.tensor(np.random.permutation(permuted_X[:, idx]), dtype=torch.float32)
        permuted_loss = criterion(model(permuted_X), y_true).item()
        feature_importance.append((feature, baseline_loss - permuted_loss))

    # Sort feature importance in descending order
    feature_importance.sort(key=lambda x: x[1], reverse=True)

    return feature_importance

# Get feature names from the DataFrame (assuming X_train is a DataFrame)
feature_names = list(X_train.columns)

# Calculate feature importance for rides into a station
feature_importance_in = calculate_feature_importance(model, X_test_tensor, y_test_in_tensor, feature_names, criterion)

# Calculate feature importance for rides out a station
feature_importance_out = calculate_feature_importance(model, X_test_tensor, y_test_out_tensor, feature_names, criterion)

# Display feature importance
print("Feature Importance (Number of Rides In ):")
for feature, importance in feature_importance_in:
    print(f"{feature}: {importance}")

print("\nFeature Importance (Number of Rides Out ):")
for feature, importance in feature_importance_out:
    print(f"{feature}: {importance}")


This is the feature importance for the baseline model, this has been done to meet he milestone requirement. However, we have not used this feature importance for any tangible outcome. This is to show a postivie correation between day of the week and the rides takes. Other fearures considered are hour_of_week.

### Feature Importance (Number of Rides In):
- **Year**: 0.0 (insignificant impact on predictions for rides in). Permuting or changing the values of the year feature didn't significantly affect the model's prediction for the number of rides in.
- **Day of Week**: -0.045 (minor impact on predictions for rides in). The model's performance decreased by approximately 0.045, indicating a slight impact on predictions.
- **Month**: -0.477 (relatively important for predicting rides in). The model's performance decreased by approximately 0.477, implying that the month feature has more influence on predictions compared to the year or day of the week for rides into the station.
- **Hour of Day**: -1.015 (most influential feature for predicting rides in). The hour of the day is the most influential feature among the listed features, with a decrease in model performance of around 1.015. This suggests that the hour of the day significantly impacts the predictions for rides into the station.

### Feature Importance (Number of Rides Out):
- **Year**: 0.0 (insignificant impact on predictions for rides out). Similar to the rides in scenario, the year feature's importance is calculated as 0.0, indicating it doesn't significantly affect predictions for the number of rides out.
- **Day of Week**: -0.045 (minor impact on predictions for rides out). Similar to rides in, the day of the week feature has a minor impact of approximately -0.045 on predictions for rides out of the station.
- **Month**: -0.457 (relatively important for predicting rides out). Like in the rides in scenario, the month feature is relatively important for predicting rides out, with a decrease in model performance of approximately -0.457.
- **Hour of Day**: -1.007 (most influential feature for predicting rides out). Similar to rides in, the hour of the day is the most influential feature for predicting the number of rides out, with a decrease in model performance of around -1.007.


The above values are based on the change in the model's performance(loss) when each feature is randomly permuted,suggesting their relative significane in making predictions for the number of rides into or out of the station.

As in the second part of the our analysis we used the STGCN model we have implemeted the clustered spatial and temporal features that are lattitues and longitutes and station names/ station ids. We have used a window for 30 mins for predicting net rides to those stations. As the model uses  only these 4 features to predict the net rides. These are the only features considered. This A STGCN model doen't not have a feature importance matrix as all the features are used to train.

In [ ]:

# Function to calculate feature importance using random permutations
def calculate_feature_importance(model, X, y_true, feature_names, criterion, n_permutations=30):
    baseline_loss = criterion(model(X), y_true).item()
    feature_importance = []

    for idx, feature in enumerate(feature_names):
        permuted_X = X.clone()
        permuted_X[:, idx] = torch.tensor(np.random.permutation(permuted_X[:, idx]), dtype=torch.float32)
        permuted_loss = criterion(model(permuted_X), y_true).item()
        feature_importance.append((feature, baseline_loss - permuted_loss))

    # Sort feature importance in descending order
    feature_importance.sort(key=lambda x: x[1], reverse=True)

    return feature_importance

# Get feature names from the DataFrame (assuming X_train is a DataFrame)
feature_names = list(X_train.columns)

# Calculate feature importance for rides into a station
feature_importance_in = calculate_feature_importance(model, X_test_tensor, y_test_in_tensor, feature_names, criterion)

# Calculate feature importance for rides out a station
feature_importance_out = calculate_feature_importance(model, X_test_tensor, y_test_out_tensor, feature_names, criterion)

# Display feature importance
print("Feature Importance (Number of Rides In ):")
for feature, importance in feature_importance_in:
    print(f"{feature}: {importance}")

print("\nFeature Importance (Number of Rides Out ):")
for feature, importance in feature_importance_out:
    print(f"{feature}: {importance}")

In [ ]:
import torch
import numpy as np

def calculate_feature_importance(model, X, y_true, criterion, n_permutations=30):
    baseline_loss = criterion(model(X), y_true).item()
    feature_importance = []

    for idx in range(X.shape[1]):
        permuted_losses = []
        for _ in range(n_permutations):
            permuted_X = X.clone()
            permuted_X[:, idx] = torch.tensor(np.random.permutation(permuted_X[:, idx]), dtype=torch.float32)
            permuted_loss = criterion(model(permuted_X), y_true).item()
            permuted_losses.append(permuted_loss)

        average_permuted_loss = np.mean(permuted_losses)
        feature_importance.append((idx, baseline_loss - average_permuted_loss))

    # Sort feature importance in descending order
    feature_importance.sort(key=lambda x: x[1], reverse=True)

    return feature_importance

# Assuming X_train and X_test_tensor are defined elsewhere
# Assuming model and criterion are defined elsewhere

# Get feature names from the number of columns in X_train
feature_names = [f'feature_{i}' for i in range(X_train.shape[1])]

# Convert X_train to a PyTorch tensor
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)

# Convert X_test to a PyTorch tensor
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

# Assuming y_train and y_test are defined and converted to PyTorch tensors
# Calculate feature importance for rides into a station
feature_importance_in = calculate_feature_importance(model, X_test_tensor, y_test_in_tensor, criterion)

# Calculate feature importance for rides out a station
feature_importance_out = calculate_feature_importance(model, X_test_tensor, y_test_out_tensor, criterion)

# Map indices to feature names for readability
feature_importance_in_named = [(feature_names[idx], importance) for idx, importance in feature_importance_in]
feature_importance_out_named = [(feature_names[idx], importance) for idx, importance in feature_importance_out]

# Display feature importance
print("Feature Importance (Number of Rides In):")
for feature, importance in feature_importance_in_named:
    print(f"{feature}: {importance}")

print("\nFeature Importance (Number of Rides Out):")
for feature, importance in feature_importance_out_named:
    print(f"{feature}: {importance}")
